##### Copyright 2018 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Automatically rewrite TF 1.x and compat.v1 API symbols

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/migrate/upgrade">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/migrate/upgrade.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/migrate/upgrade.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/migrate/upgrade.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />
    Download notebook</a>
  </td>
</table>


TensorFlow 2.x includes many API changes from TF 1.x and the `tf.compat.v1` APIs, such as reordering arguments, renaming symbols, and changing default values for parameters. Manually performing all of these modifications would be tedious and prone to error. To streamline the changes, and to make your transition to TF 2.x as seamless as possible, the TensorFlow team has created the `tf_upgrade_v2` utility to help transition legacy code to the new API.

Note: `tf_upgrade_v2` is installed automatically for TensorFlow 1.13 and later (including all TF 2.x builds).

Typical usage is like this:

<pre class="devsite-terminal devsite-click-to-copy prettyprint lang-bsh">
tf_upgrade_v2 \
  --intree my_project/ \
  --outtree my_project_v2/ \
  --reportfile report.txt
</pre>

It will accelerate your upgrade process by converting existing TensorFlow 1.x Python scripts to TensorFlow 2.x.

The conversion script automates many mechanical API transformations, though many APIs cannot be automatically migrated. It is also not able to fully make your code compatible with TF2 behaviors and APIs. So, it is only a part of your migration journey.

## Compatibility modules

Certain API symbols can not be upgraded simply by using a string replacement. Those that cannot be automatically upgraded will be mapped to their locations in the `compat.v1` module. This module replaces TF 1.x symbols like `tf.foo` with the equivalent `tf.compat.v1.foo` reference. If you are already using `compat.v1` APIs by importing TF via `import tensorflow.compat.v1 as tf`, the `tf_upgrade_v2` script will attempt to convert these usages to the non-compat APIs where possible. Note that while some `compat.v1` APIs are compatible with TF2.x behaviors, many are not. Therefore, it's recommended to manually proofread replacements and migrate them to new APIs in the `tf.*` namespace instead of `tf.compat.v1` namespace as quickly as possible.

Because of TensorFlow 2.x module deprecations (for example, `tf.flags` and `tf.contrib`), some changes can not be worked around by switching to `compat.v1`. Upgrading this code may require using an additional library (for example, [`absl.flags`](https://github.com/abseil/abseil-py)) or switching to a package in [tensorflow/addons](http://www.github.com/tensorflow/addons).


## Recommended upgrade process

The rest of this guide demonstrates how to use the symbol-rewriting script. While the script is easy to use, it is strongly recomended that you use the script as part of the following process: 

1. **Unit Test**: Ensure that the code you’re upgrading has a unit test suite with reasonable coverage. This is Python code, so the language won’t protect you from many classes of mistakes. Also ensure that any dependency you have has already been upgraded to be compatible with TensorFlow 2.x.

1. **Install TensorFlow 1.15**: Upgrade your TensorFlow to the latest TensorFlow 1.x version, at least 1.15. This includes the final TensorFlow 2.0 API in `tf.compat.v2`.

1. **Test With 1.15**: Ensure your unit tests pass at this point. You’ll be running them repeatedly as you upgrade so starting from green is important.

1. **Run the upgrade script**: Run `tf_upgrade_v2` on your entire source tree, tests included. This will upgrade your code to a format where it only uses symbols available in TensorFlow 2.0. Deprecated symbols will be accessed with `tf.compat.v1`. These will eventually require manual attention, but not immediately.

1. **Run the converted tests with TensorFlow 1.15**: Your code should still run fine in TensorFlow 1.15. Run your unit tests again. Any error in your tests here means there’s a bug in the upgrade script. [Please let us know](https://github.com/tensorflow/tensorflow/issues).

1. **Check the upgrade report for warnings and errors**: The script writes a report file that explains any conversions you should double-check, or any manual action you need to take. For example: Any remaining instances of contrib will require manual action to remove. Please consult [the RFC for more instructions](https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md). 

1. **Install TensorFlow 2.x**: At this point it should be safe to switch to TensorFlow 2.x binaries, even if you are running with legacy behaviors

1. **Test with `v1.disable_v2_behavior`**: Re-running your tests with a `v1.disable_v2_behavior()` in the tests' main function should give the same results as running under 1.15.

1. **Enable V2 Behavior**: Now that your tests work using the TF2 binaries, you can now begin migrating your code to avoiding `tf.estimator`s and only using supported TF2 behaviors (with no TF2 behavior disabling). See the [Migration guides](https://tensorflow.org/guide/migrate) for details.

## Using the symbol-rewriting `tf_upgrade_v2` script


### Setup

Before getting started ensure that TensorFlow 2.x is installed.

In [2]:
import tensorflow as tf

print(tf.__version__)

2.7.1


Clone the [tensorflow/models](https://github.com/tensorflow/models) git repository so you have some code to test on:

In [3]:
!git clone --branch r1.13.0 --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:  32% (896/2768)
Updating files:  33% (914/2768)
Updating files:  34% (942/2768)
Updating files:  35% (969/2768)
Updating files:  36% (997/2768)
Updating files:  37% (1025/2768)
Updating files:  38% (1052/2768)
Updating files:  39% (1080/2768)
Updating files:  40% (1108/2768)
Updating files:  41% (1135/2768)
Updating files:  42% (1163/2768)
Updating files:  43% (1191/2768)
Updating files:  44% (1218/2768)
Updating files:  45% (1246/2768)
Updating files:  46% (1274/2768)
Updating files:  47% (1301/2768)
Updating files:  48% (1329/2768)
Updating files:  49% (1357/2768)
Updating files:  50% (1384/2768)
Updating files:  51% (1412/2768)
Updating files:  52% (1440/2768)
Updating files:  53% (1468/2768)
Updating files:  54% (1495/2768)
Updating files:  55% (1523/2768)
Updating files:  56% (1551/2768)
Updating files:  57% (1578/2768)
Updating files:  58% (1606/2768)
Updating files:  59% (1634/2768)
Updating files:  60% (1661/2768)
Updating files:  61% (1

### Read the help

The script should be installed with TensorFlow. Here is the builtin help:

In [4]:
!tf_upgrade_v2 -h

usage: tf_upgrade_v2 [-h] [--infile INPUT_FILE] [--outfile OUTPUT_FILE]
                     [--intree INPUT_TREE] [--outtree OUTPUT_TREE]
                     [--copyotherfiles COPY_OTHER_FILES] [--inplace]
                     [--no_import_rename] [--no_upgrade_compat_v1_import]
                     [--reportfile REPORT_FILENAME] [--mode {DEFAULT,SAFETY}]
                     [--print_all]

Convert a TensorFlow Python file from 1.x to 2.0

Simple usage:
  tf_upgrade_v2.py --infile foo.py --outfile bar.py
  tf_upgrade_v2.py --infile foo.ipynb --outfile bar.ipynb
  tf_upgrade_v2.py --intree ~/code/old --outtree ~/code/new

optional arguments:
  -h, --help            show this help message and exit
  --infile INPUT_FILE   If converting a single file, the name of the file to
                        convert
  --outfile OUTPUT_FILE
                        If converting a single file, the output filename.
  --intree INPUT_TREE   If converting a whole tree of files, the directory to
        

### Example TF1 code

Here is a simple TensorFlow 1.0 script:

In [5]:
!head -n 65 models/samples/cookbook/regression/custom_regression.py | tail -n 10

'head' is not recognized as an internal or external command,
operable program or batch file.


With TensorFlow 2.x installed it does not run:

In [6]:
!(cd models/samples/cookbook/regression && python custom_regression.py)

'python' is not recognized as an internal or external command,
operable program or batch file.


### Single file

The script can be run on a single Python file:

In [7]:
!tf_upgrade_v2 \
  --infile models/samples/cookbook/regression/custom_regression.py \
  --outfile /tmp/custom_regression_v2.py

INFO line 38:8: Renamed 'tf.feature_column.input_layer' to 'tf.compat.v1.feature_column.input_layer'

Traceback (most recent call last):
  File "C:\Users\olaab\AppData\Local\Programs\Python\Python37\lib\shutil.py", line 557, in move
    os.rename(src, real_dst)
FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\olaab\\AppData\\Local\\Temp\\tmp2b_kjig3' -> '/tmp/custom_regression_v2.py'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\olaab\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\olaab\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\olaab\AppData\Local\Programs\Python\Python37\Scripts\tf_upgrade_v2.exe\__main__.py", line 7, in <module>
  File "C:\Users\olaab\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\tools\compatibility\tf_upgrade_v2_main.py", line 164, in main
    args.input_file, output_file, 


INFO line 43:10: Renamed 'tf.layers.dense' to 'tf.compat.v1.layers.dense'
INFO line 46:17: Renamed 'tf.layers.dense' to 'tf.compat.v1.layers.dense'
INFO line 57:17: tf.losses.mean_squared_error requires manual check. tf.losses have been replaced with object oriented versions in TF 2.0 and after. The loss function calls have been converted to compat.v1 for backward compatibility. Please update these calls to the TF 2.0 versions.
INFO line 57:17: Renamed 'tf.losses.mean_squared_error' to 'tf.compat.v1.losses.mean_squared_error'
INFO line 61:15: Added keywords to args of function 'tf.shape'
INFO line 62:15: Changed tf.to_float call to tf.cast(..., dtype=tf.float32).
INFO line 65:40: Renamed 'tf.train.AdamOptimizer' to 'tf.compat.v1.train.AdamOptimizer'
INFO line 68:39: Renamed 'tf.train.get_global_step' to 'tf.compat.v1.train.get_global_step'
INFO line 83:9: tf.metrics.root_mean_squared_error requires manual check. tf.metrics have been replaced with object oriented versions in TF 2.0 and

The script will print errors if it can not find a fix for the code. 

### Directory tree

Typical projects, including this simple example, will use much more than one file. Typically want to update an entire package, so the script can also be run on a directory tree:

In [8]:
# update the .py files and copy all the other files to the outtree
!tf_upgrade_v2 \
    --intree models/samples/cookbook/regression/ \
    --outtree regression_v2/ \
    --reportfile tree_report.txt

WARNING line 125:15: Changing dataset.make_one_shot_iterator() to tf.compat.v1.data.make_one_shot_iterator(dataset). Please check this transformation.

INFO line 38:8: Renamed 'tf.feature_column.input_layer' to 'tf.compat.v1.feature_column.input_layer'
INFO line 43:10: Renamed 'tf.layers.dense' to 'tf.compat.v1.layers.dense'
INFO line 46:17: Renamed 'tf.layers.dense' to 'tf.compat.v1.layers.dense'
INFO line 57:17: tf.losses.mean_squared_error requires manual check. tf.losses have been replaced with object oriented versions in TF 2.0 and after. The loss function calls have been converted to compat.v1 for backward compatibility. Please update these calls to the TF 2.0 versions.
INFO line 57:17: Renamed 'tf.losses.mean_squared_error' to 'tf.compat.v1.losses.mean_squared_error'
INFO line 61:15: Added keywords to args of function 'tf.shape'
INFO line 62:15: Changed tf.to_float call to tf.cast(..., dtype=tf.float32).
INFO line 65:40: Renamed 'tf.train.AdamOptimizer' to 'tf.compat.v1.train.Ad

Note the one warning about the `dataset.make_one_shot_iterator` function.

Now the script works in with TensorFlow 2.x:

Note that because the `tf.compat.v1` module is included in TF 1.15, the converted script will also run in TensorFlow 1.15.

In [9]:
!(cd regression_v2 && python custom_regression.py 2>&1) | tail

'tail' is not recognized as an internal or external command,
operable program or batch file.


## Detailed report

The script also reports a list of detailed changes. In this example it found one possibly unsafe transformation and included a warning at the top of the file: 

In [10]:
!head -n 20 tree_report.txt

'head' is not recognized as an internal or external command,
operable program or batch file.


Note again the one warning about the `Dataset.make_one_shot_iterator function`.

In other cases the output will explain the reasoning for non-trivial changes:

In [11]:
%%writefile dropout.py
import tensorflow as tf

d = tf.nn.dropout(tf.range(10), 0.2)
z = tf.zeros_like(d, optimize=False)

Writing dropout.py


In [12]:
!tf_upgrade_v2 \
  --infile dropout.py \
  --outfile dropout_v2.py \
  --reportfile dropout_report.txt > /dev/null

The system cannot find the path specified.


In [13]:
!cat dropout_report.txt

'cat' is not recognized as an internal or external command,
operable program or batch file.


Here is the modified file contents, note how the script adds argument names to deal with moved and renamed arguments:

In [14]:
!cat dropout_v2.py

'cat' is not recognized as an internal or external command,
operable program or batch file.


A larger project might contain a few errors. For example convert the deeplab model:

In [15]:
!tf_upgrade_v2 \
    --intree models/research/deeplab \
    --outtree deeplab_v2 \
    --reportfile deeplab_report.txt > /dev/null

The system cannot find the path specified.


It produced the output files:

In [16]:
!ls deeplab_v2

'ls' is not recognized as an internal or external command,
operable program or batch file.


But there were errors. The report will help you pin-point what you need to fix before this will run. Here are the first three errors:

In [17]:
!cat deeplab_report.txt | grep -i models/research/deeplab | grep -i error | head -n 3

'cat' is not recognized as an internal or external command,
operable program or batch file.


## "Safety" mode

The conversion script also has a less invasive `SAFETY` mode that simply changes the imports to use the `tensorflow.compat.v1` module:

In [18]:
!cat dropout.py

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [19]:
!tf_upgrade_v2 --mode SAFETY --infile dropout.py --outfile dropout_v2_safe.py > /dev/null

The system cannot find the path specified.


In [20]:
!cat dropout_v2_safe.py

'cat' is not recognized as an internal or external command,
operable program or batch file.


As you can see this doesn't upgrade your code, but does allow TensorFlow 1 code to run against TensorFlow 2 binaries. Note that this does not mean your code is running supported TF 2.x behaviors!

## Caveats

- Do not update parts of your code manually before running this script. In particular, functions that have had reordered arguments like `tf.math.argmax` or `tf.batch_to_space` cause the script to incorrectly add keyword arguments that mismap your existing code.

- The script assumes that `tensorflow` is imported using `import tensorflow as tf`, or `import tensorflow.compat.v1 as tf`.

- This script does not reorder arguments. Instead, the script adds keyword arguments to functions that have their arguments reordered.

- Check out [tf2up.ml](https://github.com/lc0/tf2up) for a convenient tool to upgrade Jupyter
  notebooks and Python files in a GitHub repository.

To report upgrade script bugs or make feature requests, please file an issue on [GitHub](https://github.com/tensorflow/tensorflow/issues).